In [5]:

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib as mpl
import pandas as pd

import os
import sys


In [ ]:

# Get the current working directory
current_dir = os.getcwd()

# Navigate to 'network_simulations'
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Navigate to the sibling directory '00_opinion_function_setup'
sibling_dir = os.path.join(parent_dir, "00_opinion_function_setup")

# Add the sibling directory to sys.path for importing
if sibling_dir not in sys.path:
    sys.path.append(sibling_dir)

# Import the module from 00_opinion_function_setup
import opinion_functions as fun  
import generate_homophilic_graph_symmetric

In [14]:

hvec=[0,0.1,0.5,0.75,0.9,1]
num_swaps=333
num_sims=10
minority_fraction=0.33
num_agents=1000
m=5
mean_misp_vec=np.zeros((num_sims,len(hvec),num_swaps))
opinion_percent_vec=np.zeros((num_sims,len(hvec),num_swaps))
majority_opinion_percent_vec=np.zeros((num_sims,len(hvec),num_swaps))
minority_opinion_percent_vec=np.zeros((num_sims,len(hvec),num_swaps))
minority_degree_fraction_vec=np.zeros((num_sims,len(hvec),num_swaps))

In [15]:
for j, h in enumerate(hvec):
    h=hvec[j]
    for k in range(num_sims):
        # if k % 10 == 0:
        print(k)
        G,minority_nodes = generate_homophilic_graph_symmetric.homophilic_ba_graph(N=num_agents, m=m, minority_fraction=minority_fraction, homophily=h)

        for i in range(num_swaps):
            G,new_minority_nodes=fun.swap_top_maj_opinion(G,minority_nodes)
            # print(minority_nodes)
            # print(new_minority_nodes)
            true_opinion,perceived_opinion=fun.generate_perceived_opinion(G,new_minority_nodes,{},narcissistic=False,weigh_connected=False)
            mean_misp_vec[k,j,i]=0.66-np.sum(perceived_opinion)/num_agents

            minority_opinion = [perceived_opinion[i] for i in minority_nodes]
            minority_misperception = [(1-minority_fraction)-perceived_opinion[i] for i in minority_nodes]
            majority_opinion = [perceived_opinion[x] for x in np.arange(len(perceived_opinion)) if x not in minority_nodes]
            majority_misperception = [(1-minority_fraction)-perceived_opinion[x] for x in np.arange(len(perceived_opinion)) if x not in minority_nodes]

            perceived_opinion_percent = [i * 100 for i in perceived_opinion]
            majority_opinion_percent=[i * 100 for i in majority_opinion]
            minority_opinion_percent=[i * 100 for i in minority_opinion]

            opinion_percent_vec[k,j,i]=(np.mean(perceived_opinion_percent))
            majority_opinion_percent_vec[k,j,i]=(np.mean(majority_opinion_percent))
            minority_opinion_percent_vec[k,j,i]=np.mean(minority_opinion_percent)
            minority_degree_fraction_vec[k,j,i]=fun.degree_fraction(G,minority_nodes)[0]

mean_misp_vec_sims=np.average(mean_misp_vec,axis=0)
mean_opinion_percent_vec=np.average(opinion_percent_vec,axis=0)
mean_majority_opinion_percent_vec=np.average(majority_opinion_percent_vec,axis=0)
mean_minority_opinion_percent_vec=np.average(minority_opinion_percent_vec,axis=0)
mean_minority_degree_fraction_vec=np.average(minority_degree_fraction_vec,axis=0)


0
1
2
3
4
5
6
7
8
9


In [17]:
for j, h in enumerate(hvec):
    # Create the dataframe with hvec as the index
    df = pd.DataFrame({
        'mean_misp': mean_misp_vec_sims[j,:],
        'mean_opinion_percent': mean_opinion_percent_vec[j,:],
        'mean_majority_opinion_percent': mean_majority_opinion_percent_vec[j,:],
        'mean_minority_opinion_percent': mean_minority_opinion_percent_vec[j,:],
        'mean_minority_degree_fraction': mean_minority_degree_fraction_vec[j,:]
    }, index=np.arange(num_swaps))

    # Rename the index to 'swap_count'
    df.index.name = 'swap_count'
    # Write the DataFrame to a CSV file
    df.to_csv(f'swapped_data/swap_sim_h_{str(int(h*100))}.csv')